# Assignment 2

Deadline: 26.03.2025, 12:00 CET

<Add your name, student-id and emal address>
* Zihan Liu, 22-736-938, zihan.liu@uzh.ch
* Yujie Tao, 23-744-378, yujie.tao@uzh.ch
* Weijie Chen, 23-743-727, weijie.chen@uzh.ch
* Schmidli Adrian, 17-728-833, adrian.schmidli@uzh.ch

## 1. Linearization of Turnover

**(15 points)**

Turnover constraints are used to limit the amount of change in portfolio weights between periods, helping to manage transaction costs and maintain portfolio stability.

Your task is to implement a method `linearize_turnover_constraint` for the class `QuadraticProgram`, which modifies the quadratic programming problem to incorporate a linearized turnover constraint. This will involve updating the objective function coefficients, equality and inequality constraints, as well as the lower and upper bounds of the problem. 

Additionally, complete the example provided below to demonstrate that your method functions correctly.

In class, we discussed a solution that involved augmenting the dimensionality by a factor of three. Here, you are asked to implement an alternative method that involves a two-fold increase in dimensions. If you are unable to implement the two-fold method, you may proceed with the three-fold approach.

### Function Parameters:
- `x_init` (np.ndarray): The initial portfolio weights.
- `to_budget` (float, optional): The maximum allowable turnover. Defaults to `float('inf')`.

### Steps for Function Implementation:

As discussed in the lecture, introduce auxiliary variables and augment the matrices/vectors used for optimization.

- **Objective Function Coefficients**:  
  Pad the existing objective function coefficients (`P` and `q`) to accommodate the new variables introduced by the turnover constraint.  
  *Note*: "Padding" refers to adding extra elements (typically zeros) to an array or matrix to increase its size to a desired shape.

- **Equality Constraints**:  
  Pad the existing equality constraint matrix (`A`) to account for the new variables.

- **Inequality Constraints**:  
  Pad the existing inequality constraint matrix (`G`) and vector (`h`) and further add a new inequality constraint row to incorporate the turnover constraint.  

- **Lower and Upper Bounds**:  
  Pad the existing lower (`lb`) and upper (`ub`) bounds to accommodate the new variables.

- **Update Problem Data**:  
  Overwrite the original problem data in the `QuadraticProgram` class with the updated matrices and vectors to include the linearized turnover constraint.

In [1]:
# Import standard libraries
import types
import os
import sys

# Import third-party libraries
import numpy as np
import pandas as pd

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))  # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course\\src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.quadratic_program import QuadraticProgram
from helper_functions import load_data_msci

In [13]:
def linearize_turnover_constraint(self, x_init: np.ndarray, to_budget=float('inf')) -> None:
        '''
        Linearize the turnover constraint in the quadratic programming problem.

        This method modifies the quadratic programming problem to include a linearized turnover constraint.

        Parameters:
        -----------
        x_init : np.ndarray
            The initial portfolio weights.
        to_budget : float, optional
            The maximum allowable turnover. Defaults to float('inf').

        Notes:
        ------
        - The self parameter in Python is used in object-oriented programming (OOP). It refers to the instance of the class where the method is being defined (here the class "QuadraticProgram").
        - None: The function doesn't return anything.
        - The method updates the problem's objective function coefficients, inequality constraints,
        equality constraints, and bounds to account for the turnover constraint.
        - The original problem data is overridden with the updated matrices and vectors.

        Examples:
        ---------
        >>> qp = QuadraticProgram(P, q, G, h, A, b, lb, ub, solver='cvxopt')
        >>> qp.linearize_turnover_constraint(x_init=np.array([0.1, 0.2, 0.3]), to_budget=0.05)
        '''
        P = self.problem_data.get('P')
        q = self.problem_data.get('q')
        A = self.problem_data.get('A')
        b = self.problem_data.get('b')
        G = self.problem_data.get('G')
        h = self.problem_data.get('h')
        lb = self.problem_data.get('lb')
        ub = self.problem_data.get('ub')
       
        # Dimensions
        n = len(q)

        # Update the coefficients of the objective function (two-fold approach)
        P_aug = np.pad(P, ((0, n), (0, n)), mode='constant')
        q_aug = np.pad(q, ((0, n)), mode='constant')

        # Update the equality constraints (two-fold approach) 
        if A is not None:
            A_aug = np.pad(A, ((0, 0), (0, n)), mode='constant')
        else:
            A_aug = A
        
        # Add additional equality constraints (two-fold approach)
        A_add_con_1 = np.zeros(2 * n)
        A_add_con_2 = np.zeros(2 * n)
        A_add_con_3 = np.zeros(2 * n)
        # A_add_con_1[n + c] = value -> add some additional constraints like a net change constraint (z1 + z2 + z3 = 0 if n = 3) 
        # ...
        # A_add_con_2[n + c] = value -> or that if one asset changes another has to change twice as much (2 * z1 - z2 = 0)
        # ...
        # A_add_con_1[n + c] = value -> etc.
        # ...
        # It is only necessary to define the z-values, that are unequal to zero. The others are set to zero automatically.
        
        A_add_con_list = []

        for name, con in locals().items():
            if name.startswith("A_add_con_") and isinstance(con, np.ndarray):
                if not np.all(con == 0):
                    A_add_con_list.append(con)

        A_add_cons = np.vstack(A_add_con_list) if A_add_con_list else None

        if A_add_cons is not None:
            if A_aug is not None:
                A_aug = np.vstack([A_aug, A_add_cons])
            else:
                A_aug = A_add_cons.reshape(len(A_add_cons), -1) 
        
        if A_add_cons is not None:
            b_add_cons = np.zeros(len(A_add_cons))
            # b_add_cons[n] = value -> can be used if we would like to add a value in the b-vector which is unequal to zero for the additional constraints.
            if b is not None:
                b_aug = np.hstack([b, b_add_cons])
            else:
                b_aug = A_add_cons.reshape(len(A_add_cons), -1)
        else:
            b_aug = b

        # Update the inequality constraints (two-fold approach)
        G_add_con_list = []

        if G is not None:
            G_aug = np.pad(G, ((0, 0), (0, n)), mode='constant')
            for i in range(n):
                row = np.zeros(2 * n)
                row[i] = 1
                row[i + n] = -1
                G_add_con_list.append(row)
                neg_row = row.copy()
                neg_row[i] = -1
                neg_row[i + n] = -1
                G_add_con_list.append(neg_row)
            row_tau = np.hstack([np.zeros(n), np.ones(n)])
            G_aug = np.vstack([G_aug, G_add_con_list, row_tau])
        else:
            for i in range(n):
                row = np.zeros(2 * n)
                row[i] = 1
                row[i + n] = -1
                G_add_con_list.append(row)
                neg_row = row.copy()
                neg_row[i] = -1
                neg_row[i + n] = -1
                G_add_con_list.append(neg_row)
            row_tau = np.hstack([np.zeros(n), np.ones(n)])
            G_aug = np.vstack([G_add_con_list, row_tau])
        
        if h is not None:
            x_init_adj = np.empty((2 * len(x_init),), dtype=float)
            x_init_adj[0::2] = x_init
            x_init_adj[1::2] = -x_init
            h_aug = np.hstack([h, x_init_adj, to_budget])
        else:
            x_init_adj = np.empty((2 * len(x_init),), dtype=float)
            x_init_adj[0::2] = x_init
            x_init_adj[1::2] = -x_init
            h_aug = np.hstack([x_init_adj, to_budget])

        # Update lower and upper bounds (two-fold approach)
        if lb is not None:
            lb_aug = np.pad(lb, ((0, n)), mode='constant')
        else:
            lb_aug = lb

        if ub is not None:
            ub_aug = np.pad(ub, (0, n), mode='constant', constant_values=np.inf)
        else:
            ub_aug = ub

        # Override the original matrices (notice: b does not change) -> have a look on my notes!
        self.update_problem_data({
            'P': P_aug,
            'q': q_aug,
            'A': A_aug,
            'b': b_aug,
            'G': G_aug,
            'h': h_aug,
            'lb': lb_aug,
            'ub': ub_aug
        })

        # For debugging
        # print("A_aug:\n", A_aug) 
        # print("b_aug:\n", b_aug)
        # print("G_aug:\n", G_aug)
        # print("h_aug:\n", h_aug)
        # print("lb_aug:\n", lb_aug)
        # print("ub_aug:\n", ub_aug)

        return None

## Demo

#### Create P and q

In [14]:
# Load the msci country index data
N = 10
data = load_data_msci(path = '../data/', n = N)
X = data['return_series']
print("Variable X =", X)

# Compute the vector of expected returns (mean returns)
q = ExpectedReturn(method='geometric').estimate(X=X, inplace=False)
print("Variable q =", q)

# Compute the covariance matrix
P = Covariance(method='pearson').estimate(X=X, inplace=False)
print("Variable P =", P)

# q, P

Variable X =                   AT        AU        BE        CA        CH        DE  \
Index                                                                    
1999-01-01  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1999-01-04  0.010057  0.009080  0.042147  0.013070  0.035885  0.052249   
1999-01-05  0.013661 -0.010048  0.020162  0.021940  0.012016  0.001444   
1999-01-06  0.000000  0.015264 -0.000078  0.027640  0.015335  0.036205   
1999-01-07  0.004104  0.016564 -0.016877 -0.003480 -0.011902 -0.020187   
...              ...       ...       ...       ...       ...       ...   
2023-04-12 -0.002488  0.004714 -0.012072  0.001708  0.000017  0.002662   
2023-04-13  0.001297 -0.003806 -0.000049  0.004728  0.000913  0.002400   
2023-04-14  0.009533  0.004829  0.007622  0.001950  0.008387  0.005797   
2023-04-17 -0.005665  0.003404  0.004917  0.003392 -0.001485 -0.002086   
2023-04-18  0.004836 -0.003262 -0.007127  0.002161  0.004373  0.006089   

                  DK    

### Create some constraints, instantiate an object of class QuadraticProgram, and add the method linearize_turnover_constraint to the instance.

In [15]:
# Instantiate the constraints with only the budget and long-only constraints (no inequality constraints)
constraints = Constraints(ids = X.columns.tolist())
constraints.add_budget(rhs=1, sense='=')
constraints.add_box(lower=0.0, upper=1.0)
GhAb = constraints.to_GhAb()

# Create a quadratic program and linearize the turnover constraint
qp = QuadraticProgram(
    P = P.to_numpy(),
    q = q.to_numpy() * 0,
    G = GhAb['G'],
    h = GhAb['h'],
    A = GhAb['A'],
    b = GhAb['b'],
    lb = constraints.box['lower'].to_numpy(),
    ub = constraints.box['upper'].to_numpy(),
    solver = 'cvxopt',
)

# Add the linearized turnover constraint method to the instance of class QuadraticProgram
qp.linearize_turnover_constraint = types.MethodType(linearize_turnover_constraint, qp)

### Add a turnover limit of 50%. Solve the problem and check whether the turnover constraint is respected.

In [16]:
# Prepare initial weights
x_init = pd.Series([1/X.shape[1]]*X.shape[1], index=X.columns)

# Add the linearized turnover constraint
qp.linearize_turnover_constraint(x_init=x_init, to_budget=0.5)

# Solve the problem
qp.solve()

# Check the turnover
solution = qp.results.get('solution')
ids = constraints.ids
weights = pd.Series(solution.x[:len(ids)], index=ids)

# print("Weights:", x_init)
# print("Solution:", weights)
print("Turnover:")
print(np.abs(weights - x_init).sum())

Turnover:
0.4995454132549856
